In [5]:
import setGPU

import sys
sys.path.append('../')

# import torch
from escnn import gspaces
from escnn import group
from escnn import nn

import equinox as eqx
import jax
import jax.numpy as jnp
import numpy as np
import torch
from jaxtyping import Array, Float, Int, PyTree, PRNGKeyArray  # https://github.com/google/jaxtyping

from torch.utils.data import Dataset
from torchvision.transforms import RandomRotation
from torchvision.transforms import Pad
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
from torchvision.transforms import Compose
from torchvision.transforms import InterpolationMode

import numpy as np

from PIL import Image

In [2]:
# download the dataset
!wget -nc http://www.iro.umontreal.ca/~lisa/icml2007data/mnist_rotation_new.zip
# uncompress the zip file
!unzip -n mnist_rotation_new.zip -d mnist_rotation_new

class MnistRotDataset(Dataset):
    
    def __init__(self, mode, transform=None):
        assert mode in ['train', 'test']
            
        if mode == "train":
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_train_valid.amat"
        else:
            file = "mnist_rotation_new/mnist_all_rotation_normalized_float_test.amat"
        
        self.transform = transform

        data = np.loadtxt(file, delimiter=' ')
            
        self.images = data[:, :-1].reshape(-1, 28, 28).astype(np.float32)
        self.labels = data[:, -1].astype(np.int64)
        self.num_samples = len(self.labels)
    
    def __getitem__(self, index):
        image, label = self.images[index], self.labels[index]
        image = Image.fromarray(image, mode='F')
        if self.transform is not None:
            image = self.transform(image)
        return image, label
    
    def __len__(self):
        return len(self.labels)

# images are padded to have shape 29x29.
# this allows to use odd-size filters with stride 2 when downsampling a feature map in the model
pad = Pad((0, 0, 1, 1), fill=0)

# to reduce interpolation artifacts (e.g. when testing the model on rotated images),
# we upsample an image by a factor of 3, rotate it and finally downsample it again
resize1 = Resize(87)
resize2 = Resize(29)

totensor = ToTensor()

File ‘mnist_rotation_new.zip’ already there; not retrieving.

Archive:  mnist_rotation_new.zip


In [3]:
train_transform = Compose([
    pad,
    resize1,
    RandomRotation(180., interpolation=InterpolationMode.BILINEAR, expand=False),
    resize2,
    totensor,
])

mnist_train = MnistRotDataset(mode='train', transform=train_transform)
train_loader = torch.utils.data.DataLoader(mnist_train, batch_size=64)


test_transform = Compose([
    pad,
    totensor,
])
mnist_test = MnistRotDataset(mode='test', transform=test_transform)
test_loader = torch.utils.data.DataLoader(mnist_test, batch_size=64)

x, y = mnist_train[0]
x = jnp.array(x[None, ...])
y = jnp.array(y[None, ...])
print(x.shape)
print(y.shape)

(1, 1, 29, 29)
(1,)


In [4]:
class C8SteerableCNN(eqx.Module):
    layers: list
    fully_net: list
    input_type: nn.FieldType

    def __init__(self, key, n_classes=10):
        keys = jax.random.split(key, 8)
        layers = []

        super(C8SteerableCNN, self).__init__()
        
        # the model is equivariant under rotations by 45 degrees, modelled by C8
        r2_act = gspaces.rot2dOnR2(N=8)
        
        # the input image is a scalar field, corresponding to the trivial representation
        in_type = nn.FieldType(r2_act, [r2_act.trivial_repr])
        
        # we store the input type for wrapping the images into a geometric tensor during the forward pass
        self.input_type = in_type
        
        # convolution 1
        # first specify the output type of the convolutional layer
        # we choose 24 feature fields, each transforming under the regular representation of C8
        out_type = nn.FieldType(r2_act, 24*[r2_act.regular_repr])
        layers.extend([
            nn.MaskModule(in_type, 29, margin=1),
            nn.R2Conv(in_type, out_type, kernel_size=7, padding=1, use_bias=False, key=keys[0]),
            # nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        ])
        print("conv 1")
        
        # convolution 2
        # the old output type is the input type to the next layer
        # in_type = self.block1.out_type
        in_type = out_type
        # the output type of the second convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 48*[r2_act.regular_repr])
        layers.extend([
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[1]),
            # nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        ])
        print("conv 2")
        layers.append(nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2))
        print("PointwiseAvgPoolAntialiased")
        
        # convolution 3
        # the old output type is the input type to the next layer
        # in_type = self.block2.out_type
        in_type = out_type
        # the output type of the third convolution layer are 48 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 48*[r2_act.regular_repr])
        layers.extend([
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[2]),
            # nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        ])
        print("conv 3")
        
        # convolution 4
        # the old output type is the input type to the next layer
        # in_type = self.block3.out_type
        in_type = out_type
        # the output type of the fourth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 96*[r2_act.regular_repr])
        layers.extend([
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[3]),
            # nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        ])
        print("conv 4")
        layers.append(
            nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=2)
        )
        print("PointwiseAvgPoolAntialiased")
        
        # convolution 5
        # the old output type is the input type to the next layer
        # in_type = self.block4.out_type
        in_type = out_type
        # the output type of the fifth convolution layer are 96 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 96*[r2_act.regular_repr])
        layers.extend([
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=2, use_bias=False, key=keys[4]),
            # nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        ])
        print("conv 5")
        
        # convolution 6
        # the old output type is the input type to the next layer
        # in_type = self.block5.out_type
        in_type = out_type
        # the output type of the sixth convolution layer are 64 regular feature fields of C8
        out_type = nn.FieldType(r2_act, 64*[r2_act.regular_repr])
        layers.extend([
            nn.R2Conv(in_type, out_type, kernel_size=5, padding=1, use_bias=False, key=keys[5]),
            # nn.InnerBatchNorm(out_type),
            nn.ReLU(out_type)
        ])
        print("conv 5")
        layers.append(nn.PointwiseAvgPoolAntialiased(out_type, sigma=0.66, stride=1, padding=0))
        print("PointwiseAvgPoolAntialiased")
        
        layers.append(nn.GroupPooling(out_type))
        print("GroupPooling")

        self.layers = layers
        
        # number of output channels
        # c = self.gpool.out_type.size
        c = out_type.size
        print("c", c)
        c = 64
        print("c", c)
        
        # Fully Connected
        self.fully_net = [
            eqx.nn.Linear(c, 64, key=keys[6]),
            # eqx.nn.BatchNorm() .BatchNorm1d(64),
            jax.nn.elu,
            eqx.nn.Linear(64, n_classes, key=keys[7]),
            jax.nn.log_softmax
        ]
        print("Sequential")
    
    def __call__(self, input: Array):
        # wrap the input tensor in a GeometricTensor
        # (associate it with the input type)
        x = nn.GeometricTensor(input, self.input_type)
        
        # apply each equivariant block
        for layer in self.layers:
            # print("x", x.shape)
            x = layer(x)

        # unwrap the output GeometricTensor
        # (take the Pytorch tensor and discard the associated representation)
        x = x.tensor
        # print("x", x.shape)
        x = x.reshape(x.shape[0], -1)
        
        # classify with the final fully connected layers)
        # x = self.fully_net(x.reshape(x.shape[0], -1))
        for layer in self.fully_net:
            # print("x", x.shape)
            x = jax.vmap(layer)(x)

        return x

In [28]:
BATCH_SIZE = 64
LEARNING_RATE = 3e-4
STEPS = 300
PRINT_EVERY = 30
SEED = 5678

key = jax.random.PRNGKey(SEED)
x, y = mnist_train[0]
x, y = jnp.array(x[None, ...]), jnp.array(y[None, ...])
print(x.shape, y.shape)

key, subkey = jax.random.split(key, 2)
# model = C8SteerableCNN(subkey)
# eqx.filter_jit(model)(x)

g = gspaces.no_base_space(group.so2_group(7))
reprs = [g.irrep(*irr) for irr in g.fibergroup.bl_irreps(3)] + [g.fibergroup.bl_regular_representation(3)]
r1 = g.type(*reprs)
r2 = g.type(*reprs)
model = nn.Linear(r1, r2, key=subkey)
print(model)

import optax
optim = optax.adamw(LEARNING_RATE)
opt_state = optim.init(eqx.filter(model, eqx.is_array))
print("opt_state", opt_state)
params, static = eqx.partition(model, eqx.is_array)
# print("params", params)
# print("static", static)

def loss(
    model, x
) -> Float[Array, ""]:
    pred_y = model(x).tensor
    return (pred_y ** 2).sum()

input = nn.GeometricTensor(jax.random.normal(key, (4, r1.size)), r1)

def loss2(params, static, x):
    model = eqx.combine(params, static)
    return loss(model, x)

loss_value, grads = jax.value_and_grad(loss2)(params, static, input)
print(loss_value)

# This will work too!
value, grads = eqx.filter_value_and_grad(loss)(model, input)
print(value)
print("grads", grads)


(1, 1, 29, 29) (1,)
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis END
SteerableKernelBasis BEG
SteerableKernelBasis END
Linear(
  in_type=[SO(2): {irr

In [26]:
grads

Linear(
  in_type=[SO(2): {irrep_0 (x1), irrep_1 (x1), irrep_2 (x1), irrep_3 (x1), regular_3 (x1)}(14)],
  out_type=[SO(2): {irrep_0 (x1), irrep_1 (x1), irrep_2 (x1), irrep_3 (x1), regular_3 (x1)}(14)],
  space=SO(2),
  weights=f32[28],
  bias=f32[2],
  bias_expansion=f32[14,2],
  expanded_bias=f32[14],
  matrix=f32[14,14],
  _basisexpansion=<escnn.nn.modules.basismanager.basisexpansion_blocks.BlocksBasisExpansion object at 0x7eff84129220>,
  inference=None
)

In [30]:
def test_model(model: eqx.Module, x: Image):
    np.set_printoptions(linewidth=10000)
    
    # evaluate the `model` on 8 rotated versions of the input image `x`
    # model = model.eval()
    
    x = resize1(pad(x))
    
    print()
    print('##########################################################################################')
    header = 'angle |  ' + '  '.join(["{:6d}".format(d) for d in range(10)])
    print(header)
    # with torch.no_grad():
    for r in range(8):
        x_transformed = totensor(resize2(x.rotate(r*45., Image.BILINEAR))).reshape(1, 1, 29, 29)
        # print("x_transformed", type(x_transformed), x_transformed)
        x_transformed = jnp.array(x_transformed.numpy())#.to(device)

        y = model(x_transformed)
        # y = y.to('cpu').numpy().squeeze()
        y = np.array(y).squeeze()
        
        angle = r * 45
        print("{:5d} : {}".format(angle, y))
    print('##########################################################################################')
    print()

In [31]:
# build the test set    
raw_mnist_test = MnistRotDataset(mode='test')
# retrieve the first image from the test set
x, y = next(iter(raw_mnist_test))

# evaluate the model
test_model(model, x)


##########################################################################################
angle |       0       1       2       3       4       5       6       7       8       9
    0 : [-2.3537765 -2.48019   -2.6330705 -2.4628375 -2.165155  -2.0853043 -2.5181222 -2.591998  -2.042986  -1.965477 ]
   45 : [-2.3421402 -2.4550412 -2.6289797 -2.4677906 -2.1908054 -2.1059937 -2.517116  -2.5953166 -2.058967  -1.9334837]
   90 : [-2.3446524 -2.4242864 -2.6203306 -2.4772458 -2.1802363 -2.0924573 -2.5060844 -2.5675755 -2.0983903 -1.9557058]
  135 : [-2.334875  -2.4323788 -2.6082888 -2.4587193 -2.185335  -2.089982  -2.5209122 -2.573486  -2.0848866 -1.9730031]
  180 : [-2.3537765 -2.4801898 -2.6330705 -2.4628375 -2.165155  -2.0853043 -2.5181227 -2.591998  -2.042986  -1.965477 ]
  225 : [-2.34214   -2.4550414 -2.6289797 -2.4677908 -2.1908052 -2.1059937 -2.5171163 -2.595317  -2.058967  -1.9334838]
  270 : [-2.3446524 -2.4242861 -2.6203303 -2.4772456 -2.1802363 -2.0924573 -2.5060842 -2.5675752 -2.

In [32]:
def loss(
    # model, x: Float[Array, "batch 1 28 28"], y: Int[Array, " batch"]
    model, x: Float[Array, "batch 1 29 29"], y: Int[Array, " batch"]
) -> Float[Array, ""]:
    # Our input has the shape (BATCH_SIZE, 1, 28, 28), but our model operations on
    # a single input input image of shape (1, 28, 28).
    #
    # Therefore, we have to use jax.vmap, which in this case maps our model over the
    # leading (batch) axis.
    # pred_y = jax.vmap(model)(x)
    pred_y = model(x)
    return cross_entropy(y, pred_y)


def cross_entropy(
    y: Int[Array, " batch"], pred_y: Float[Array, "batch 10"]
) -> Float[Array, ""]:
    # y are the true targets, and should be integers 0-9.
    # pred_y are the log-softmax'd predictions.
    pred_y = jnp.take_along_axis(pred_y, jnp.expand_dims(y, 1), axis=1)
    return -jnp.mean(pred_y)


# Example loss
x, y = mnist_train[0]
x, y = jnp.array(x[None, ...]), jnp.array(y[None, ...])
loss_value = loss(model, x, y)
print(loss_value.shape, loss_value)  # scalar loss
# Example inference
# output = jax.vmap(model)(x)
# output = jax.jit(model)(x)
output = model(x)


# This will work!
# params, static = eqx.partition(model, eqx.is_array)


# def loss2(params, static, x, y):
#     model = eqx.combine(params, static)
#     return loss(model, x, y)


# loss_value, grads = jax.value_and_grad(loss2)(params, static, x, y)
# print(loss_value)

# This will work too!
value, grads = eqx.filter_value_and_grad(loss)(model, x, y)
print(value)

() 2.5231252
2.5231252


In [33]:
loss = eqx.filter_jit(loss)  # JIT our loss function from earlier!


@eqx.filter_jit
def compute_accuracy(
    model: CNN, x: Float[Array, "batch 1 28 28"], y: Int[Array, " batch"]
) -> Float[Array, ""]:
    """This function takes as input the current model
    and computes the average accuracy on a batch.
    """
    # pred_y = jax.vmap(model)(x)
    pred_y = model(x)
    pred_y = jnp.argmax(pred_y, axis=1)
    return jnp.mean(y == pred_y)

def evaluate(model: CNN, testloader: torch.utils.data.DataLoader):
    """This function evaluates the model on the test dataset,
    computing both the average loss and the average accuracy.
    """
    avg_loss = 0
    avg_acc = 0
    for x, y in testloader:
        x = jnp.array(x.numpy())
        y = jnp.array(y.numpy())
        # Note that all the JAX operations happen inside `loss` and `compute_accuracy`,
        # and both have JIT wrappers, so this is fast.
        avg_loss += loss(model, x, y)
        avg_acc += compute_accuracy(model, x, y)
    return avg_loss / len(testloader), avg_acc / len(testloader)

evaluate(model, test_loader)

(Array(2.346044, dtype=float32), Array(0.0972866, dtype=float32))

In [34]:
import optax
optim = optax.adamw(LEARNING_RATE)

def train(
    model: CNN,
    trainloader: torch.utils.data.DataLoader,
    testloader: torch.utils.data.DataLoader,
    optim: optax.GradientTransformation,
    steps: int,
    print_every: int,
) -> CNN:
    # Just like earlier: It only makes sense to train the arrays in our model,
    # so filter out everything else.
    opt_state = optim.init(eqx.filter(model, eqx.is_array))

    # Always wrap everything -- computing gradients, running the optimiser, updating
    # the model -- into a single JIT region. This ensures things run as fast as
    # possible.
    # @eqx.filter_jit
    def make_step(
        model: CNN,
        opt_state: PyTree,
        # x: Float[Array, "batch 1 28 28"],
        x: Float[Array, "batch 1 29 29"],
        y: Int[Array, " batch"],
    ):
        loss_value, grads = eqx.filter_value_and_grad(loss)(model, x, y)
        updates, opt_state = optim.update(grads, opt_state, model)
        model = eqx.apply_updates(model, updates)
        return model, opt_state, loss_value

    # Loop over our training dataset as many times as we need.
    def infinite_trainloader():
        while True:
            yield from train_loader

    for step, (x, y) in zip(range(steps), infinite_trainloader()):
        # PyTorch dataloaders give PyTorch tensors by default,
        # so convert them to NumPy arrays.
        x = jnp.array(x.numpy())
        y = jnp.array(y.numpy())
        model, opt_state, train_loss = make_step(model, opt_state, x, y)
        if (step % print_every) == 0 or (step == steps - 1):
            test_loss, test_accuracy = evaluate(model, testloader)
            print(
                f"{step=}, train_loss={train_loss.item()}, "
                f"test_loss={test_loss.item()}, test_accuracy={test_accuracy.item()}"
            )
    return model

In [35]:
model = train(model, train_loader, test_loader, optim, STEPS, PRINT_EVERY)

step=0, train_loss=2.3170299530029297, test_loss=2.3205418586730957, test_accuracy=0.09728660434484482
step=30, train_loss=1.27370023727417, test_loss=1.1153783798217773, test_accuracy=0.6206042170524597
step=60, train_loss=0.6582151651382446, test_loss=0.5333170294761658, test_accuracy=0.838934600353241
step=90, train_loss=0.31860435009002686, test_loss=0.3456519544124603, test_accuracy=0.8982976078987122
step=120, train_loss=0.3775779902935028, test_loss=0.35995593667030334, test_accuracy=0.8904252052307129
step=150, train_loss=0.21673333644866943, test_loss=0.25681743025779724, test_accuracy=0.9210757613182068
step=180, train_loss=0.1893269121646881, test_loss=0.2114839404821396, test_accuracy=0.9363011717796326
step=210, train_loss=0.15129354596138, test_loss=0.2073531299829483, test_accuracy=0.935262143611908
step=240, train_loss=0.4047985076904297, test_loss=0.29849594831466675, test_accuracy=0.9113451242446899
step=270, train_loss=0.15914244949817657, test_loss=0.158601894974708